In [3]:
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import random

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Convolution2D 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [5]:
### INIT CNN ###
vector = np.array((0,0,0))
###########
### CNN ###
###########
inputshape = vector.shape
model2 = Sequential()
model2.add(Dense(12,input_dim=3,activation='relu'))
model2.add(Dense(64,activation='relu'))
#model2.add(Dense(64,activation='relu'))
#model2.add(Dense(32,activation='relu'))
model2.add(Dense(units=1,activation='sigmoid')) #min_value=0, max_value=1
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#model2.compile(loss='CategoricalCrossentropy',optimizer='adam',metrics=['accuracy'])
#model2.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse'])
print("the prediction for vector is: ",model2.predict(vector.reshape(1,3)))

the prediction for vector is:  [[0.5]]


In [6]:
pole_length = 0.3 #[m]
angle=0
time_interval = 0.03 #[sec]
g=9.8 #[m/s^2]
ball_mass = 0.005 #[kg]
ball_radius = 0.025 #[m]
friction_coefficient = 0.2
v0 = 0
x0 = 0 #[m]
min_angle=-8 #[deg]
max_angle=-8 #[deg]
height,width=240,640
length = 610
image = np.ones((height, width)) * 255


sucess_time = 15 #[sec]
gamma = 0.9
wins = 0
loss = 0
x_train,y_train = [],[]
i_step_in_current_round = 0 
i_step_in_total_from_beginning = 0 
num_of_round = 0 



In [7]:
def predict2degree(degreeAsFloat):
    mydegree = np.round(degreeAsFloat[0]*18.6)
    return mydegree

In [8]:
def drawImage(ballPosition,lineAngle):
    #draw line
    image = np.ones((height, width)) * 255
    normalize_x = ballPosition
    angle = lineAngle
    x_center, y_center = int(width/2), int(height/2)
    x2 = x_center+int(length/2*math.cos(angle))
    y2 = y_center+int(length/2*math.sin(angle))
    x1,y1 = (x_center-int(length/2*math.cos(angle))),(y_center-int(length/2*math.sin(angle)))
    line_thickness = 2
    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), thickness=line_thickness)
    
    M = (y2-y1)/(x2-x1)
    y_center_of_circle = M*normalize_x + y1 -M*x1
    
    #draw circle
    radius = 10
    center_coordinates = (int(normalize_x),int(y_center_of_circle+radius))
    color = (0, 255, 0)
    thickness = 2
    image = cv2.circle(image, center_coordinates, radius, color, thickness)
        
    image = np.flip(image)
    return image

In [9]:
degree = predict2degree(model2.predict(vector.reshape(1,3)))
if(degree>=0.5):
    degree=5
else:
    degree=-5
#print("init degree",degree)
print("starting")
num_of_round=0
gamesToPlay=2
while(num_of_round<gamesToPlay):
    x_train,y_train = [],[]
    num_of_round+=1
    print("round number:",num_of_round)
    mystack = [np.ones((height, width)) * 255]
    #print("type(degree)",type(degree))
    for i in range(0,100):
        if(True):#random.random() < math.exp(-num_of_round/(gamesToPlay/3))):
            degree=random.random()
            print("randomize degree",degree)
            if(degree>=0.5):
                degree=5
            else:
                degree=-5
        else:
            print("degree is not randomize for this round")
        degree = float(degree)
        print("degree:",degree)
        if(((angle>0) and (degree<0)) or ((angle<0) and (degree>0))):
            if(degree<0):
                loopsign=-1
                degree=5
            else:
                loopsign=+1
                degree=-5
            for j in range(1,10):
                degree = degree+(1*loopsign)
                print("degree changing",degree)
                #change_sloop_slowly
                angle=math.pi*(degree/180)
                acceleration = (3/5)*g*math.sin(angle)
                current_v = v0 + acceleration*time_interval 
                current_x = x0 + v0*time_interval + (1/2)*acceleration*(time_interval**2)
                v0 = current_v
                x0 = current_x
                normalize_x = 300-current_x*300/1.5
                #print("after",time_interval,"[sec], at angle=",angle*180/math.pi,"[deg], the ball moved to ",current_x)
                #print("angle",angle)
                image = drawImage(normalize_x,angle)
                mystack = np.concatenate((mystack,[image]),axis=0)
        else:
            for j in range(1,10):
                angle=math.pi*(degree/180)
                acceleration = (3/5)*g*math.sin(angle)
                current_v = v0 + acceleration*time_interval 
                current_x = x0 + v0*time_interval + (1/2)*acceleration*(time_interval**2)
                v0 = current_v
                x0 = current_x
                normalize_x = 300-current_x*300/1.5
                #print("after",time_interval,"[sec], at angle=",angle*180/math.pi,"[deg], the ball moved to ",current_x)
                #print("angle",angle)
                image = drawImage(normalize_x,angle)
                mystack = np.concatenate((mystack,[image]),axis=0)
            
        #print("degree for i",i,"move is:",degree)
        angle=math.pi*(degree/180)
        
        acceleration = (3/5)*g*math.sin(angle)
        
        current_v = v0 + acceleration*time_interval 
        current_x = x0 + v0*time_interval + (1/2)*acceleration*(time_interval**2)
        
        
        v0 = current_v
        x0 = current_x
        normalize_x = 300-current_x*300/1.5
        #print("after",time_interval,"[sec], at angle=",angle*180/math.pi,"[deg], the ball moved to ",current_x)
        
        #print("angle",angle)
        image = drawImage(normalize_x,angle)
        mystack = np.concatenate((mystack,[image]),axis=0)
        
        vector = np.array((degree/5,normalize_x/600,current_v/10))
        model_prediction = model2.predict(vector.reshape(1,3))
        #print("model_prediction",model_prediction)
        if(model_prediction>=0.5):
            model_prediction=1
            degree=5
        else:
            model_prediction=0
            degree=-5
        #degree = predict2degree(model_prediction)

        x_train.append(vector)
        y_train.append(model_prediction)
    
        #print("vector",vector)
        #print("model_prediction:",model_prediction," <==> degree:",degree)
        
        
        
        if((normalize_x > 600) or (normalize_x < 10)):
            break
        if(i*time_interval >=sucess_time):
            break
           
    print("game ended after",i,"steps")          
    x0=((random.random()-0.5)*2)/5
    v0=((random.random()-0.5)*2)/10
    
    if(i*time_interval >= sucess_time):
        print("model ended without ball falling")
    elif((normalize_x > 600) or (normalize_x < 10)):
        print("game ended becouse ball dropped")
        if(len(y_train)>=12):
            for j in range(1,11,1): 
                    y_train[len(y_train)-j] = (1-y_train[len(y_train)-j])
        else:
            for j in range(1,int(len(y_train)*0.8),1): 
                    y_train[len(y_train)-j] = (1-y_train[len(y_train)-j])
   
    for j in range(random.randrange(0,int(0.2*len(y_train)))):
        numToChange =  random.randrange(0,len(y_train))
        y_train[numToChange] = (1-y_train[numToChange])
   
                   
    model2.fit(x=np.vstack(x_train),y=np.vstack(y_train))
    
    

starting
round number: 1
randomize degree 0.6786009144424052
degree: 5.0
randomize degree 0.8166403740920923
degree: 5.0
randomize degree 0.916094212668362
degree: 5.0
randomize degree 0.23525845575365667
degree: -5.0
degree changing 4
degree changing 3
degree changing 2
degree changing 1
degree changing 0
degree changing -1
degree changing -2
degree changing -3
degree changing -4
randomize degree 0.8000579298199827
degree: 5.0
degree changing -4
degree changing -3
degree changing -2
degree changing -1
degree changing 0
degree changing 1
degree changing 2
degree changing 3
degree changing 4
randomize degree 0.3964951235441847
degree: -5.0
degree changing 4
degree changing 3
degree changing 2
degree changing 1
degree changing 0
degree changing -1
degree changing -2
degree changing -3
degree changing -4
randomize degree 0.25562824360668635
degree: -5.0
randomize degree 0.18178326940952994
degree: -5.0
randomize degree 0.9056812777995965
degree: 5.0
degree changing -4
degree changing -3
d

In [10]:
import imageio; 
from IPython.display import Video; 
edges = mystack
imageio.mimwrite('test2.mp4', edges, fps=33); 
Video('test2.mp4', width=640, height=480)

Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


In [11]:
imageio.mimwrite('/tmp/simulate.mp4', edges, fps=33); 


Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


In [ ]:
x_train,y_train = [],[]
vector = np.array((4/8,100/600,6/10))
#int((model2.predict(vector.reshape(1,3))-0.5)*8)
degree = predict2degree(model2.predict(vector.reshape(1,3)))
print(degree)




In [ ]:
x_train,y_train = [],[]
print("vector:",vector)
print("degree:",degree)
x_train.append(vector)
y_train.append(-1*(degree/16+0.5))
print(y_train)
model2.fit(x=np.vstack(x_train),y=np.vstack(y_train))

In [ ]:
print(len(y_train))

In [ ]:
for j in range(0,10):
    print(j)